In [263]:
# Import requirements

##SPACY IMPORTS
import spacy

nlp = spacy.load('en_core_web_sm')
nlp.pipe_names
ner = nlp.get_pipe('ner')

import random
from spacy.util import minibatch, compounding
from pathlib import Path

##KERAS IMPORTS
import re
import torch
import torch.nn as nn

import numpy as np
import pandas as pd
import random
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
import nltk
import re
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout
from keras.callbacks import ModelCheckpoint
nltk.download("stopwords")
nltk.download("punkt")


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/maxlengdell/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/maxlengdell/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [264]:
#Load intent model
model = load_model("model.h5")

/Users/maxlengdell/opt/anaconda3/envs/nnD7046E/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [265]:
def cleaning(sentences):
    words = []
    for s in sentences:
        clean = re.sub(r'[^ a-z A-Z 0-9]', " ", s)
        w = word_tokenize(clean)
        #stemming
        words.append([i.lower() for i in w])
    
    return words  

def create_tokenizer(words, filters = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'):
    token = Tokenizer(filters = filters)
    token.fit_on_texts(words)
    #print(token)
    return token
def encoding_doc(token, words):
    return(token.texts_to_sequences(words))

def padding_doc(encoded_doc, max_length):
    return(pad_sequences(encoded_doc, maxlen = max_length, padding = "post"))

def get_max_length(words):
    return(len(max(words, key = len)))

def load_dataset(filename):
    df = pd.read_csv(filename, encoding = "latin1", names = ["Sentence", "Intent"])
    print(df.head())
    intent = df["Intent"]
    unique_intent = list(set(intent))
    sentences = list(df["Sentence"])
  
    return (intent, unique_intent, sentences)
  

intent, unique_intent, sentences = load_dataset("data.csv")

#define stemmer
stemmer = LancasterStemmer()


                    Sentence   Intent
0  How many <item> are left?  balance
1           Stock of <Item>?  balance
2        Do you have <Item>?  balance
3         Balance of <Item>?  balance
4           Any <Item> left?  balance


In [266]:
cleaned_words = cleaning(sentences)
print(len(cleaned_words))
print(cleaned_words[:2])  

118
[['how', 'many', 'item', 'are', 'left'], ['stock', 'of', 'item']]


In [267]:
word_tokenizer = create_tokenizer(cleaned_words)
vocab_size = len(word_tokenizer.word_index) + 1
max_length = get_max_length(cleaned_words)

print("Vocab Size = %d and Maximum length = %d" % (vocab_size, max_length))

Vocab Size = 82 and Maximum length = 9


In [268]:
encoded_doc = encoding_doc(word_tokenizer, cleaned_words)
padded_doc = padding_doc(encoded_doc, max_length)

output_tokenizer = create_tokenizer(unique_intent, filters = '!"#$%&()*+,-/:;<=>?@[\]^`{|}~')


In [269]:
encoded_output = encoding_doc(output_tokenizer, intent)

In [270]:
encoded_output = np.array(encoded_output).reshape(len(encoded_output), 1)

In [271]:
encoded_output.shape

(118, 1)

In [272]:
def predictions(text):
    
    clean = re.sub(r'[^ a-z A-Z 0-9]', " ", text)
    test_word = word_tokenize(clean)
    test_word = [w.lower() for w in test_word]
    test_ls = word_tokenizer.texts_to_sequences(test_word)

    #Check for unknown words
    if [] in test_ls:
        test_ls = list(filter(None, test_ls))
    
    test_ls = np.array(test_ls).reshape(1, len(test_ls))
 
    x = padding_doc(test_ls, max_length)
  
    pred = model.predict_proba(x)
  
  
    return pred
    

In [273]:
def one_hot(encode):
    o = OneHotEncoder(sparse = False)
    return(o.fit_transform(encode))

In [274]:
#output_one_hot = one_hot(encoded_output)
#output_one_hot.shape


In [275]:
#from sklearn.model_selection import train_test_split

In [276]:
def get_final_output(pred, classes):
    predictions = pred[0]
 
    classes = np.array(classes)
    ids = np.argsort(-predictions)
    classes = classes[ids]
    predictions = -np.sort(-predictions)
 
    for i in range(pred.shape[1]):
        print("%s has confidence = %s" % (classes[i], (predictions[i])))




In [277]:
text = "How much does that cost?"
pred = predictions(text)
get_final_output(pred, unique_intent)

price has confidence = 0.9999708
order has confidence = 2.7602082e-05
balance has confidence = 1.1163619e-06
common has confidence = 4.594952e-07


In [278]:
text = "Can I order 3 pants?"
pred = predictions(text)
get_final_output(pred, unique_intent)

common has confidence = 0.99798715
order has confidence = 0.0020095224
balance has confidence = 3.3716383e-06
price has confidence = 3.3261423e-09


In [279]:
text = "How many pants are left?"
pred = predictions(text)
get_final_output(pred, unique_intent)

balance has confidence = 0.9999981
common has confidence = 1.768541e-06
order has confidence = 6.638163e-08
price has confidence = 9.85351e-09


In [280]:
#Load entity model

output_dir = Path('./Models/')

print("loading model form", output_dir)
nlp_updated = spacy.load(output_dir)
doc = nlp_updated("Those pants are really good")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

loading model form Models


/Users/maxlengdell/opt/anaconda3/envs/nnD7046E/lib/python3.6/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Entities [('Those', 'ORG'), ('pants', 'PRODUCT')]


In [281]:
opening_lines = ['Hello, what can I help you with?',
                'How can I help you?',
                'Whats on your mind today?']
stock_info = [(['glove', '299', '3']), #product name, price, balance
             (['shoes', '899', '5']),
             (['pants', '450', '10'])]

def randomOpening():
    index = random.randint(0,len(opening_lines)-1)
    print(opening_lines[index])
def getProductPrice(product):
    for prod in stock_info:
        if(prod[0]==product):
            return prod[1]
def getProductBalance(product):
    for prod in stock_info:
        if(prod[0]==product):
            return prod[2]


In [286]:
def getEntity(sentence):
    
    entities = nlp_updated(sentence)
    #print("Entities", [(ent.text, ent.label_) for ent in doc.ents])  
    text = []
    label = []
    for ent in doc.ents:
        text.append(ent.text)
        label.append(ent.label_)
    ret_arr = [text,label]
    return doc.ents

In [287]:
def getIntent(sentence):
    pred = predictions(sentence)
    guess = pred[0]
    
    classes = np.array(unique_intent)
    ids = np.argsort(-pred)
    classes = classes[ids]
    predict = -np.sort(-pred)
    
    print("Network guessed:", classes[0][0])
    return classes[0][0]


In [288]:
def priceFunc(entities):
    print("Oh, so you want to know the price of your product?")
    for ent in entities:
        print(ent.label_, ent.text)
        if(ent.label_=="PRODUCT"):
            print("The price of " + ent.text + " is " + getProductPrice(ent.text))
            #print(ent.text, ent.label_)
            

def balanceFunc(entities):
    print("Do you want to know the storage count of your product?")
    for ent in entities:

        if(ent.label_=="PRODUCT"):
            print("The balance of " + ent.text + " is " + getProductBalance(ent.text))


def orderFunc(entities):
    print("Print order info")
    for ent in entities:

        if(ent.label_=="PRODUCT"):
            print("You have placed an order on " + ent.text)
            
def commonCunf(entities):
    print("I'm sorry, i did not understand. Try rephrasing the sentence.")

In [ ]:
#Ask user from the opening lines-list
#identify intent
#identify entities
#send to corresponding function and return answer

def cli():
    while True:
        randomOpening()
        user_input = input("Ask the chatbot something: ")
        if(user_input=="quit"):
            break
        else:
            intent = getIntent(user_input)
            entity = getEntity(user_input)
            print(entity)
            if(intent=="price"):
                priceFunc(entity)
            elif(intent=="balance"):
                balanceFunc(entity)
            elif(intent=="order"):
                orderFunc(entity)
                

        
cli()

How can I help you?
Ask the chatbot something: what does the shoes cost?
Network guessed: price
(Those, pants)
Oh, so you want to know the price of your product?
ORG Those
PRODUCT pants
The price of pants is 450
Hello, what can I help you with?
Ask the chatbot something: Buy 4 shoes please
Network guessed: common
(Those, pants)
Whats on your mind today?
Ask the chatbot something: buy 3 pants please
Network guessed: common
(Those, pants)
Whats on your mind today?
